get the NIDOVIRALES subset list

done

In [46]:
import pandas as pd
virus_list=pd.read_csv('VMR_MSL38_v2.csv')

In [47]:
nido_list=virus_list[virus_list['Order']=='Nidovirales'].copy(deep=True)

In [48]:
from typing import List,Dict
def robust_split(s:str,split:str)->List[str]:
    return [i.strip() for i in s.split(split)]

def get_genbank_id(accession:str)->Dict[str,str]:
    assert isinstance(accession,str),f'accession: {accession} is not a str!'
    o={}
    holder_token=0
    if ';' in accession:
        for sub_a in robust_split(accession,';'):
            if ':' in sub_a:
                subk,subv=robust_split(sub_a,':')
                o[subk]=subv
            else:
                o[f'{holder_token}']=sub_a
                holder_token+=1
    else:
        if ":" in accession:
            subk,subv=robust_split(accession,':')
            o[subk]=subv
        else:
            o['_']=accession
    return o

In [49]:
from warnings import warn
def get_correct_name(name:str)->str:
    "some names block contains multiple name sep by ';'"
    assert isinstance(name,str),f'valid input: {name}'
    if ';' in name:
        warn(f'multiple names: {name},use the first one')
        return robust_split(name,';')[0]
    else:
        return name

In [50]:
nido_list['true_access']=nido_list['Virus GENBANK accession'].apply(get_genbank_id)
nido_list['true_name']=nido_list['Virus name abbreviation(s)'].apply(get_correct_name)

# nido_list[]

In [51]:
def is_multiple_access(access:Dict[str,str])->bool:
    ''' `False`: one seg; `True`:c segs '''
    if '_' in access:
        return False
    else:
        return True

In [52]:
def get_file_stem(virus_series:pd.Series,
    access_col='true_access',name_col='true_name')->List[str]:
    _=virus_series
    access:Dict[str,str]= _[access_col]
    if not is_multiple_access(_[access_col]):
        return [f"{_[name_col]}||{access['_']}"]
    else:
        return [f"{_[name_col]}|{k}|{v}" for k,v in access.items()]

In [53]:
file_stems=[]
for _,s in nido_list.iterrows():
    file_stems.extend(get_file_stem(s))

In [55]:
from pathlib import Path
from subprocess import run
odir=Path('nido_subset')
for i in file_stems:
    gf=Path(f'scan_result/{i}:genome.json')
    sf=Path(f'scan_result/{i}:segs.json')
    if not gf.is_file():
        print(f'not found: {gf}')
    else:
        run(['ln','-s',gf.absolute(),(odir/gf.name).absolute()])
    if not sf.is_file():
        print(f'not found: {sf}')
    else:
        run(['ln','-s',sf.absolute(),(odir/sf.name).absolute()])


In [27]:
nido_list.to_csv('tmp-nido.csv',index=False)

In [8]:
len(nido_list)

135